In [ ]:
#calculating a Dummy classfier - baseline for accuracy
from sklearn.model_selection import train_test_split
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split

# Print column names to inspect
cleaned_df = pd.read_excel('SpotifyDataEngineer.xlsx')

cleaned_df


In [37]:
#Remove columns "unnamed"

unnamed_columns = [col for col in cleaned_df.columns if 'Unnamed' in col]
cleaned_df.drop(columns=unnamed_columns, inplace=True)

In [38]:
#Save the changes in the newest dataset 

cleaned_df.to_excel('SpotifyDataEngineer.xlsx')



In [39]:
print(cleaned_df.columns)

Index(['artists_num', 'album_num_tracks', 'peak_rank', 'weeks_on_chart',
       'streams', 'danceability', 'energy', 'key', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration', 'release_year', 'release_month', 'release_day',
       'release_dayofweek', 'loudness_log', 'energy_danceability_interaction'],
      dtype='object')


This enable training of the DT on training data 

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score


# Selecting relevant columns for the model
# Assuming 'streams' is the target variable and excluding highly categorical columns for simplicity
relevant_columns = ['artists_num', 'album_num_tracks', 'peak_rank', 'weeks_on_chart','streams', 'danceability', 'energy', 'key', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration', 'release_year', 'release_month', 'release_day',
'release_dayofweek', 'loudness_log', 'energy_danceability_interaction',]
data_for_model = cleaned_df[relevant_columns]


# Splitting the data into training and testing sets
X = data_for_model.drop('streams', axis=1)
y = data_for_model['streams']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initializing and training the Decision Tree Regressor
dt_regressor = DecisionTreeRegressor(max_depth=15, min_samples_split=75, min_samples_leaf=25, random_state=0)
dt_regressor.fit(X_train, y_train)

# Predicting and evaluating the model
y_pred = dt_regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse) 
r2 = r2_score(y_test, y_pred)


print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

Mean Squared Error: 2.6751889624311544e+16
Root Mean Squared Error: 163560048.98602697
R-squared: 0.15003230991604688


Lets look at how the RANDOM FORREST REGRESSOR PERFORME:

We wanna do the same approach as above but for the random forrest regression model. We are training the model on the training set, and evaluate the performance

In [43]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Initializing and training the Decision Tree Regressor
rf_regressor = RandomForestRegressor(max_depth=10, min_samples_split=60, min_samples_leaf=15, random_state=0)
rf_regressor.fit(X_train, y_train)

# Predicting and evaluating the model
y_pred = rf_regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse) 
r2 = r2_score(y_test, y_pred)


print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)


Mean Squared Error: 2.326696626405144e+16
Root Mean Squared Error: 152535131.24540013
R-squared: 0.2607561615855013
